<a href="https://colab.research.google.com/github/Ravindu26/Spam-Mail-Detector-SMD/blob/main/Spam_Mail_Detector_SMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
from scipy.sparse import hstack

Data Collection & Pre-Processing

In [ ]:
# loading the data from csv file to a pandas Dataframe
raw_my_mail_data_set = pd.read_csv('/content/my_mail_data_set.csv')

In [ ]:
print(raw_my_mail_data_set)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [ ]:
# replace the null values with a null string
my_mail_data_set = raw_my_mail_data_set.where((pd.notnull(raw_my_mail_data_set)), '')

In [ ]:
# Calculate the length of each email in terms of characters
my_mail_data_set['Email_Length_Char'] = my_mail_data_set['Message'].apply(len)
my_mail_data_set['Email_Length_Words'] = my_mail_data_set['Message'].apply(lambda x: len(x.split()))
my_mail_data_set['Email_Length_Sentences'] = my_mail_data_set['Message'].apply(lambda x: len(re.split('[!.?]', x)))

In [ ]:
# printing the first 5 rows of the dataframe
my_mail_data_set.head()

,Category,Message,Email_Length_Char,Email_Length_Words,Email_Length_Sentences
0,ham,"Go until jurong point, crazy.. Available only ...",111,20,9
1,ham,Ok lar... Joking wif u oni...,29,6,7
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,28,2
3,ham,U dun say so early hor... U c already then say...,49,11,7
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,13,1


In [ ]:
# checking the number of rows and columns in the dataframe
my_mail_data_set.shape

(5572, 5)

Label Encoding

In [ ]:
# label spam mail as 0;  ham mail as 1;

my_mail_data_set.loc[my_mail_data_set['Category'] == 'spam', 'Category',] = 0
my_mail_data_set.loc[my_mail_data_set['Category'] == 'ham', 'Category',] = 1

spam  -  0

ham  -  1

In [ ]:
# separating the data as texts and label

X = my_mail_data_set[['Message', 'Email_Length_Char', 'Email_Length_Words', 'Email_Length_Sentences']]

Y = my_mail_data_set['Category']

In [ ]:
print(X)

                                                Message  Email_Length_Char  \
0     Go until jurong point, crazy.. Available only ...                111   
1                         Ok lar... Joking wif u oni...                 29   
2     Free entry in 2 a wkly comp to win FA Cup fina...                155   
3     U dun say so early hor... U c already then say...                 49   
4     Nah I don't think he goes to usf, he lives aro...                 61   
...                                                 ...                ...   
5567  This is the 2nd time we have tried 2 contact u...                160   
5568               Will ü b going to esplanade fr home?                 36   
5569  Pity, * was in mood for that. So...any other s...                 57   
5570  The guy did some bitching but I acted like i'd...                125   
5571                         Rofl. Its true to its name                 26   

      Email_Length_Words  Email_Length_Sentences  
0           

In [ ]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


Splitting the data into training data & test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [ ]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572, 4)
(4457, 4)
(1115, 4)


Feature Extraction

In [ ]:
# transform the text data to feature vectors that can be used as input to the Logistic regression

feature_extraction = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)

X_train_features = feature_extraction.fit_transform(X_train['Message'])
X_test_features = feature_extraction.transform(X_test['Message'])

# Convert additional features to sparse matrices

X_train_additional_features = X_train[['Email_Length_Char', 'Email_Length_Words', 'Email_Length_Sentences']].values
X_test_additional_features = X_test[['Email_Length_Char', 'Email_Length_Words', 'Email_Length_Sentences']].values

# Combine TF-IDF features and additional features
X_train_features = hstack((X_train_features, X_train_additional_features))
X_test_features = hstack((X_test_features, X_test_additional_features))

# convert Y_train and Y_test values as integers

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [ ]:
print(X_train)

                                                Message  Email_Length_Char  \
3075                Don know. I did't msg him recently.                 35   
1787  Do you know why god created gap between your f...                151   
1614                       Thnx dude. u guys out 2nite?                 28   
4304                                    Yup i'm free...                 15   
3266  44 7732584351, Do you want a New Nokia 3510i c...                173   
...                                                 ...                ...   
789   5 Free Top Polyphonic Tones call 087018728737,...                157   
968   What do u want when i come back?.a beautiful n...                220   
1667  Guess who spent all last night phasing in and ...                 73   
3321  Eh sorry leh... I din c ur msg. Not sad alread...                 91   
1688  Free Top ringtone -sub to weekly ringtone-get ...                114   

      Email_Length_Words  Email_Length_Sentences  
3075        

In [ ]:
print(X_train_features)

  (0, 5413)	0.6198254967574347
  (0, 4456)	0.4168658090846482
  (0, 2224)	0.413103377943378
  (0, 3811)	0.34780165336891333
  (0, 2329)	0.38783870336935383
  (1, 4080)	0.18880584110891163
  (1, 3185)	0.29694482957694585
  (1, 3325)	0.31610586766078863
  (1, 2957)	0.3398297002864083
  (1, 2746)	0.3398297002864083
  (1, 918)	0.22871581159877646
  (1, 1839)	0.2784903590561455
  (1, 2758)	0.3226407885943799
  (1, 2956)	0.33036995955537024
  (1, 1991)	0.33036995955537024
  (1, 3046)	0.2503712792613518
  (1, 3811)	0.17419952275504033
  (2, 407)	0.509272536051008
  (2, 3156)	0.4107239318312698
  (2, 2404)	0.45287711070606745
  (2, 6601)	0.6056811524587518
  (3, 2870)	0.5864269879324768
  (3, 7414)	0.8100020912469564
  (4, 50)	0.23633754072626942
  (4, 5497)	0.15743785051118356
  :	:
  (4448, 7433)	5.0
  (4449, 7431)	33.0
  (4449, 7432)	8.0
  (4449, 7433)	3.0
  (4450, 7431)	50.0
  (4450, 7432)	10.0
  (4450, 7433)	3.0
  (4451, 7431)	38.0
  (4451, 7432)	9.0
  (4451, 7433)	1.0
  (4452, 7431)	157.

Training the  model 

Logistic Regression

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
# training the Logistic Regression model with the training data
model.fit(X_train_features, Y_train)

LogisticRegression(max_iter=1000)

Evaluating the trained model

In [ ]:
# prediction on training data

prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

In [ ]:
print('Accuracy on training data : ', accuracy_on_training_data)

Accuracy on training data :  0.9748709894547902


In [ ]:
# prediction on test data

prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

In [ ]:
print('Accuracy on test data : ', accuracy_on_test_data)

Accuracy on test data :  0.9623318385650225


Building a Predictive System

In [ ]:
input_mail = ["I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times"]

# convert text to feature vectors
input_data_features = feature_extraction.transform(input_mail)

# Calculate additional features for the input data

input_data_additional_features = np.array([
    len(input_mail[0]),
    len(input_mail[0].split()),
    len(re.split('[!.?]', input_mail[0]))
]).reshape(1, -1)

# Combine TF-IDF features and additional features for the input data
input_data_features = hstack((input_data_features, input_data_additional_features))

# making prediction

prediction = model.predict(input_data_features)
print(prediction)


if (prediction[0]==1):
  print('Ham mail')

else:
  print('Spam mail')

[1]
Ham mail
